In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
!pip install nltk
import nltk
nltk.download('punkt_tab')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Toshima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Toshima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Toshima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
emotion_df= pd.read_csv('emotion.csv')

In [3]:
emotion_df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [4]:
violence_df= pd.read_csv('violence.csv')

In [5]:
violence_df.head()

,Tweet_ID,tweet,type
0,ID_0022DWKP,Had a dream i got raped last night. By a guy i...,sexual_violence
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence


In [6]:
hate_df= pd.read_csv('hate.csv')

In [7]:
hate_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


DATA PREPROCESSING

In [8]:
#dropping unwanted columns
violence_df.drop(columns= ['Tweet_ID'], inplace=True)
hate_df = hate_df[['tweet', 'class']]

In [9]:
emotion_df.drop(columns= ['Unnamed: 0'], inplace= True)

In [10]:
emotion_df.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [11]:
violence_df.head()

,tweet,type
0,Had a dream i got raped last night. By a guy i...,sexual_violence
1,he thought the word raped means sex and told m...,sexual_violence
2,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,Chessy Prout can do better by telling the trut...,sexual_violence


In [12]:
hate_df.head()

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [13]:
emotion_df.columns, violence_df.columns, hate_df.columns

(Index(['text', 'label'], dtype='object'),
 Index(['tweet', 'type'], dtype='object'),
 Index(['tweet', 'class'], dtype='object'))

In [14]:
#renaming the column
violence_df.rename(columns = {'tweet': 'text', 'type': 'label'}, inplace= True)
hate_df.rename(columns = {'tweet': 'text', 'class': 'label'}, inplace= True)

In [15]:
emotion_df.columns, violence_df.columns, hate_df.columns

(Index(['text', 'label'], dtype='object'),
 Index(['text', 'label'], dtype='object'),
 Index(['text', 'label'], dtype='object'))

In [16]:
#checking for null values
emotion_df.isna().sum(), violence_df.isna().sum(), hate_df.isna().sum()

(text     0
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64)

In [17]:
emotion_df.shape, violence_df.shape, hate_df.shape

((416809, 2), (39650, 2), (24783, 2))

Taking (Extracting) 12 thousand rows from each dataset

In [18]:
emotion_df['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [19]:
e_df= pd.DataFrame()
for i in range(6):
    # Get the number of rows for the current label
    num_rows = len(emotion_df[emotion_df['label'] == i])

    # Sample a maximum of num_rows or 2000, whichever is smaller
    sample_size = min(num_rows, 2000)

    subset = emotion_df[emotion_df['label'] == i].sample(n=sample_size, random_state=42)
    e_df = pd.concat([e_df, subset])


In [20]:
e_df.shape

(12000, 2)

In [21]:
emotion_df= e_df.copy()

In [22]:
emotion_df['label'].value_counts()

label
0    2000
1    2000
2    2000
3    2000
4    2000
5    2000
Name: count, dtype: int64

In [23]:
violence_df['label'].value_counts()

label
sexual_violence                 32648
Physical_violence                5946
emotional_violence                651
economic_violence                 217
Harmful_Traditional_practice      188
Name: count, dtype: int64

In [24]:
#we just want 4998 rows from the sexual_violence label
sexual_violence= violence_df[violence_df['label'] == 'sexual_violence'].sample(n=4998, random_state=42)
violence_df= violence_df[violence_df['label'] != 'sexual_violence']

In [25]:
violence_df.shape

(7002, 2)

In [26]:
violence_df= pd.concat([sexual_violence, violence_df], axis=0)

In [27]:
violence_df.shape

(12000, 2)

In [28]:
hate_df['label'].value_counts()

label
1    19190
2     4163
0     1430
Name: count, dtype: int64

In [29]:
#we just want 6407 rows from the offensive_speech label
offensive_speech= hate_df[hate_df['label'] == 1].sample(n=6407, random_state=42)
hate_df= hate_df[hate_df['label'] != 1]

In [30]:
hate_df.shape

(5593, 2)

In [31]:
hate_df= pd.concat([offensive_speech, hate_df], axis=0)

In [32]:
hate_df.shape

(12000, 2)

In [33]:
emotion_df.shape, violence_df.shape, hate_df.shape

((12000, 2), (12000, 2), (12000, 2))

In [34]:
emotion_df.head(3)

,text,label
133243,ive learned to surround myself with women who ...,0
88501,i already feel crappy because of this and you ...,0
131379,i feel like i have lost mourned and moved past...,0


In [35]:
violence_df.head(3)

,text,label
35209,My cousin was raped by this guy Matthew. She w...,sexual_violence
38191,HAHAHAHAHAHAHHA I DIDN’T SEE IT THE FIRST TIME...,sexual_violence
20925,I remember how I almost got raped like it was ...,sexual_violence


In [36]:
hate_df.head(3)

,text,label
22889,Why is it everytime I go to cracker barrel the...,1
20565,"Run that nigga, you don't want that nigga, but...",1
10780,I need a girl from Jamaica I can't fuck with t...,1


In [37]:
#resetting the indexes
emotion_df.reset_index(drop=True, inplace=True)
violence_df.reset_index(drop=True, inplace=True)
hate_df.reset_index(drop=True, inplace=True)

In [38]:
emotion_df.head(3)

,text,label
0,ive learned to surround myself with women who ...,0
1,i already feel crappy because of this and you ...,0
2,i feel like i have lost mourned and moved past...,0


In [39]:
violence_df.head(3)

,text,label
0,My cousin was raped by this guy Matthew. She w...,sexual_violence
1,HAHAHAHAHAHAHHA I DIDN’T SEE IT THE FIRST TIME...,sexual_violence
2,I remember how I almost got raped like it was ...,sexual_violence


In [40]:
hate_df.head(3)

,text,label
0,Why is it everytime I go to cracker barrel the...,1
1,"Run that nigga, you don't want that nigga, but...",1
2,I need a girl from Jamaica I can't fuck with t...,1


LABEL ENCODING

In [41]:
label_encoder= LabelEncoder()
violence_df['label']= label_encoder.fit_transform(violence_df['label'])

In [42]:
violence_df.head()

,text,label
0,My cousin was raped by this guy Matthew. She w...,4
1,HAHAHAHAHAHAHHA I DIDN’T SEE IT THE FIRST TIME...,4
2,I remember how I almost got raped like it was ...,4
3,He raped me 👈,4
4,"A woman raped by A male: psychological horror,...",4


In [43]:
violence_df['label'].unique()

array([4, 1, 3, 0, 2])

Stopwords Removal

In [44]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Toshima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Toshima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
#loading the stopwords
stop_words= set(stopwords.words('english'))

In [46]:
len(stop_words)

198

In [47]:
#stopwords removal function

def remove_stopwords(text):
    all_words = nltk.word_tokenize(text)
    filtered_words = [word for word in all_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

emotion_df['text'] = emotion_df['text'].apply(remove_stopwords)
violence_df['text'] = violence_df['text'].apply(remove_stopwords)
hate_df['text'] = hate_df['text'].apply(remove_stopwords)

In [48]:
emotion_df.head(3)

,text,label
0,ive learned surround women lift leave feeling ...,0
1,already feel crappy upset situation doesnt help,0
2,feel like lost mourned moved past tears relati...,0


Tokenization and Padding

In [49]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([emotion_df['text'], violence_df['text'], hate_df['text']]))

In [50]:
emotion_sequences = tokenizer.texts_to_sequences(emotion_df['text'])
violence_sequences = tokenizer.texts_to_sequences(violence_df['text'])
hate_sequences = tokenizer.texts_to_sequences(hate_df['text'])

In [51]:
emotion_df['text'].iloc[2]

'feel like lost mourned moved past tears relationship'

In [52]:
emotion_sequences[2:3]

[[1, 5, 321, 11854, 1207, 422, 1093, 385]]

In [53]:
max_length = 50
emotion_padded = pad_sequences(emotion_sequences, maxlen=max_length, padding='post')
violence_padded = pad_sequences(violence_sequences, maxlen=max_length, padding='post')
hate_padded = pad_sequences(hate_sequences, maxlen=max_length, padding='post')

In [54]:
emotion_padded[2:3]

array([[    1,     5,   321, 11854,  1207,   422,  1093,   385,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]])

In [55]:
#generating labels in numpy array format
emotion_labels = np.array(emotion_df['label'])
violence_labels = np.array(violence_df['label'])
hate_labels = np.array(hate_df['label'])

Model Definition

In [56]:
#prepare separate inputs for each dataset
emotion_input = emotion_padded
violence_input = violence_padded
hate_input = hate_padded

In [57]:
#defining multiple input layers for each task
emotion_input_layer = keras.layers.Input(shape=(max_length,), name='emotion_input')
violence_input_layer = keras.layers.Input(shape=(max_length,), name='violence_input')
hate_input_layer = keras.layers.Input(shape= (max_length,), name= 'hate_input')

In [58]:
#use as shared embedding layer
embedding_layer = keras.layers.Embedding(input_dim = len(tokenizer.word_index) +1, output_dim = 128)


In [59]:
#apply the embedding layer to each input
emotion_embedding = embedding_layer(emotion_input_layer)
violence_embedding = embedding_layer(violence_input_layer)
hate_embedding = embedding_layer(hate_input_layer)

In [60]:
#shared LSTM layer
shared_lstm= keras.layers.LSTM(64, return_sequences= True)

In [61]:
emotion_lstm = shared_lstm(emotion_embedding)
violence_lstm = shared_lstm(violence_embedding)
hate_lstm = shared_lstm(hate_embedding)

In [62]:
#shared global average pooling layer and dropout layer
shared_pooling = keras.layers.GlobalAveragePooling1D()
shared_dropout = keras.layers.Dropout(0.5)

In [63]:
emotion_features= shared_dropout(shared_pooling(emotion_lstm))
violence_features= shared_dropout(shared_pooling(violence_lstm))
hate_features= shared_dropout(shared_pooling(hate_lstm))

In [64]:
len(emotion_df['label'].unique()), len(violence_df['label'].unique()), len(hate_df['label'].unique())

(6, 5, 3)

In [65]:
#output layers
emotion_output = keras.layers.Dense(6, activation='softmax', name='emotion_output')(emotion_features)
violence_output = keras.layers.Dense(5, activation='softmax', name='violence_output')(violence_features)
hate_output = keras.layers.Dense(3, activation='softmax', name='hate_output')(hate_features)

In [66]:
#compile the model with multiple inputs and outputs
model= keras.Model(inputs= [emotion_input_layer, violence_input_layer, hate_input_layer],
                   outputs= [emotion_output, violence_output, hate_output])

In [67]:
model.compile(optimizer='adam',
              loss= {
                  'emotion_output' : 'sparse_categorical_crossentropy',
                  'violence_output' : 'sparse_categorical_crossentropy',
                  'hate_output' : 'sparse_categorical_crossentropy'
              },
              metrics= {
                  'emotion_output' : 'accuracy',
                  'violence_output' : 'accuracy',
                  'hate_output' : 'accuracy'
              })

In [68]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ emotion_input (InputLayer)    │ (None, 50)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ violence_input (InputLayer)   │ (None, 50)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ hate_input (InputLayer)       │ (None, 50)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 50, 128)           │       5,350,656 │ emotion_input[0][0],       │
│                               │                           │                 │ violence_input[0][0],      │
│                               │                           │                 │ hate_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 50, 64)            │          49,408 │ embedding[0][0],           │
│                               │                           │                 │ embedding[1][0],           │
│                               │                           │                 │ embedding[2][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 64)                │               0 │ lstm[0][0], lstm[1][0],    │
│ (GlobalAveragePooling1D)      │                           │                 │ lstm[2][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 64)                │               0 │ global_average_pooling1d[… │
│                               │                           │                 │ global_average_pooling1d[… │
│                               │                           │                 │ global_average_pooling1d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ emotion_output (Dense)        │ (None, 6)                 │             390 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ violence_output (Dense)       │ (None, 5)                 │             325 │ dropout[1][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ hate_output (Dense)           │ (None, 3)                 │             195 │ dropout[2][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 5,400,974 (20.60 MB)

 Trainable params: 5,400,974 (20.60 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Assuming 'violence_labels' is original label array
violence_labels = np.clip(violence_labels, 0, 4)
# This line clips any values outside the range 0-4 to the boundaries.

# Now, when you fit the model:
model.fit(x= {'emotion_input' : emotion_input,
              'violence_input' : violence_input,
              'hate_input' : hate_input},
          y= {'emotion_output' : emotion_labels,
              'violence_output' : violence_labels, # Using the clipped labels
              'hate_output' : hate_labels},
          epochs= 10,
          batch_size= 4)

Epoch 1/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 401s 130ms/step - emotion_output_accuracy: 0.3716 - emotion_output_loss: 1.4899 - hate_output_accuracy: 0.7592 - hate_output_loss: 0.6334 - loss: 2.3454 - violence_output_accuracy: 0.9273 - violence_output_loss: 0.2221
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 391s 130ms/step - emotion_output_accuracy: 0.8377 - emotion_output_loss: 0.5279 - hate_output_accuracy: 0.8752 - hate_output_loss: 0.3613 - loss: 0.9189 - violence_output_accuracy: 0.9936 - violence_output_loss: 0.0296
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 340s 113ms/step - emotion_output_accuracy: 0.9337 - emotion_output_loss: 0.2228 - hate_output_accuracy: 0.9243 - hate_output_loss: 0.2145 - loss: 0.4511 - violence_output_accuracy: 0.9967 - violence_output_loss: 0.0138
Epoch 4/10
2429/3000 ━━━━━━━━━━━━━━━━━━━━ 1:06 117ms/step - emotion_output_accuracy: 0.9609 - emotion_output_loss: 0.1183 - hate_output_accuracy: 0.9669 - hate_output_loss: 0.1069 - loss: 0.2301 - violence_output_accu

Prediction and Evaluation

In [ ]:
prediction = model.predict({'emotion_input' : emotion_input,
                            'violence_input' : violence_input,
                            'hate_input' : hate_input})

In [ ]:
prediction

In [ ]:
emotion_pred = np.argmax(prediction[0], axis=1)
violence_pred = np.argmax(prediction[1], axis=1)
hate_pred = np.argmax(prediction[2], axis=1)

In [ ]:
violence_df['label'].unique()

In [ ]:
def plot_cm(true, pred, title, labels):
  cf= confusion_matrix(true, pred, normalize = 'true')
  plt.figure(figsize = (7,6))
  sns.heatmap(cf, annot= True, cmap= 'Blues', xticklabels= labels, yticklabels= labels)
  plt.title(title)
  plt.xlabel('Predicted')
  plt.ylabel('Actual')

emotion_labels_text = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
violence_labels_text = ['sexual_violence', 'physical_violence', 'emotional_violence', 'Harmful_traditional_practice','economic_violence']
hate_labels_text = ['offensive_speech', 'Neither', 'Hate_Speech']

In [ ]:
plot_cm(emotion_labels, emotion_pred, 'Confusion Matrix for Emotion', emotion_labels_text)
plot_cm(violence_labels, violence_pred, 'Confusion Matrix for Violence', violence_labels_text)
plot_cm(hate_labels, hate_pred, 'Confusion Matrix for Hate', hate_labels_text)

Manual Testing

In [ ]:
def classify_text(input_text):
  #preprocess the input text
  input_text_cleaned = remove_stopwords(input_text)
  input_sequence = tokenizer.texts_to_sequences([input_text_cleaned])
  input_padded = pad_sequences(input_sequence, maxlen=max_length, padding='post')

  #prediction
  predictions = model.predict({'emotion_input' : input_padded,
                              'violence_input' : input_padded,
                              'hate_input' : input_padded})
  emotion_pred = np.argmax(predictions[0], axis=1)[0]
  violence_pred = np.argmax(predictions[1], axis=1)[0]
  hate_pred = np.argmax(predictions[2], axis=1)[0]

  #determine major label
  major_labels = ['Emotion', 'Violence', 'Hate']
  major_label_index = np.argmax([np.max(predictions[0]), np.max(predictions[1]), np.max(predictions[2])])
  major_label = major_labels[major_label_index]

  #determine sub labels
  emotion_labels_text = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
  violence_labels_text = ['sexual_violence', 'physical_violence', 'emotional_violence', 'Harmful_traditional_practice','economic_violence']
  hate_labels_text = ['offensive_speech', 'Neither', 'Hate_Speech']

  if major_label == 'Emotion':
    sub_label = emotion_labels_text[emotion_pred]
  elif major_label == 'Violence':
    sub_label = violence_labels_text[violence_pred]
  else:
    sub_label = hate_labels_text[hate_pred]

  return major_label, sub_label


In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
#define a text widget and a placeholder
input_text_widget = widgets.Text(
    description = 'Input text : ',
    placeholder = 'Enter your text'
)

In [ ]:
#define classify button
button= widgets.Button(description = 'classify')

In [ ]:
#define an output area to display result
output= widgets.Output()

In [ ]:
#function to handle event
def on_button_click(b):
  with output:
    output.clear_output()   #clear the previous output
    input_text = input_text_widget.value
    major_label, sub_label = classify_text(input_text)
    print(f'Major Label : {major_label}')
    print(f'Sub Label : {sub_label}')

#attach the button at bottom
button.on_click(on_button_click)

In [ ]:
display(input_text_widget, button, output)